<a href="https://colab.research.google.com/github/ounospanas/KD_PULSE/blob/main/notebooks/KD_student_dalia_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import os
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

from sklearn.preprocessing import StandardScaler

In [2]:
torch.__version__

'2.1.0+cu121'

In [3]:
window = 8
batch_size = 256

In [4]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# assumes that data have been stored to gdrive using the load_segment_data_.ipynb
ppg_X = np.load('/content/drive/MyDrive/ppg_data/dalia_data.npy')
y = np.load('/content/drive/MyDrive/ppg_data/dalia_y.npy')
subs = np.load('/content/drive/MyDrive/ppg_data/dalia_subs.npy')
acts = np.load('/content/drive/MyDrive/ppg_data/dalia_acts.npy')

In [6]:
val_sets = [[5,6,7,8],[9,10,11,12],[13,14,15],[1,2,3,4]]

In [7]:
def defineSets(X, y, subs, v, t):
    tests = []
    vals = []

    tests_b = []
    vals_b = []

    y_tests = []
    y_vals = []

    all_data = list(np.unique(subs))
    [all_data.remove(i) for i in v]

    ts= X[subs == t].reshape(-1,1,4,32*window)
    val = X[np.in1d(subs, [i for i in v if i!=t])].reshape(-1,1,4,32*window)

    y_ts= y[subs == t]
    y_val = y[np.in1d(subs, [i for i in v if i!=t])]

    tr = X[np.in1d(subs, all_data)].reshape(-1,1,4,32*window)
    y_tr = y[np.in1d(subs, all_data)]

    return tr, val, ts, y_tr, y_val, y_ts

In [8]:
tr, vals, tests, y_tr, y_vals, y_tests = defineSets(ppg_X, y, subs, val_sets[0], val_sets[0][0])

In [9]:
def z_score(
    train_s,
    val_s,
    test_s,
):
    train_signal = np.copy(train_s)
    val_signal = np.copy(val_s)
    test_signal = np.copy(test_s)

    for i in range(train_signal.shape[2]):

        mean = torch.mean(torch.from_numpy(train_signal[:, :, i, :]))
        std = torch.std(torch.from_numpy(train_signal[:, :, i, :]))

        print(mean,std)

        train_signal[:, :, i, :] = (torch.from_numpy(train_signal[:, :, i, :]) - mean) / std
        val_signal[:, :, i, :] = (torch.from_numpy(val_signal[:, :, i, :]) - mean) / std
        test_signal[:, :, i, :] = (torch.from_numpy(test_signal[:, :, i, :]) - mean) / std

    x_train = train_signal
    x_val = val_signal
    x_test = test_signal

    return x_train, x_val, x_test

In [10]:
class ReadyData(Dataset):

    def __init__(self, X, y, scale_X=False):
        if not torch.is_tensor(X):
            if scale_X:
                for i in range(4):
                    X[:,0,i,:] = StandardScaler().fit_transform(X[:,0,i,:]) #batch z-score
            self.X = torch.Tensor(X)
        if not torch.is_tensor(y):
            self.y = torch.Tensor(y)


    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Model

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
from time import time

In [12]:
# exact same RConvPPG
class Teacher(nn.Module):
    def __init__(self, conv_blocks=3, conv_layers=3, in_channels=[1,32,48], out_channels=[32,48,64],
             kernel_size=(1,5), dilation=2, padding=(0,4), dropout=[0.5,0.5,0.5], pooling_size=[(1,4),(1,2),(1,2)],
              heads=4, dim=16, dense_out=32, ppg_channels=1):
        super(Teacher, self).__init__()

        # hyperparameters
        self.conv_blocks = conv_blocks
        self.conv_layers =conv_layers
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.padding = padding
        self.dropout = dropout
        self.pooling_size = pooling_size
        self.heads = heads
        self.dim = dim
        self.dense_out = dense_out
        self.ppg_channels = ppg_channels


        self.conv11 = nn.Conv2d(in_channels=self.in_channels[0], out_channels=self.out_channels[0],
                                kernel_size=self.kernel_size, dilation=self.dilation, padding=self.padding)
        self.conv12 = nn.Conv2d(in_channels=self.out_channels[0], out_channels=self.out_channels[0],
                                kernel_size=self.kernel_size, dilation=self.dilation, padding=self.padding)
        self.conv13 = nn.Conv2d(in_channels=self.out_channels[0], out_channels=self.out_channels[0],
                                kernel_size=self.kernel_size, dilation=self.dilation, padding=self.padding)

        self.conv21 = nn.Conv2d(in_channels=self.in_channels[1], out_channels=self.out_channels[1],
                                kernel_size=self.kernel_size, dilation=self.dilation, padding=self.padding)
        self.conv22 = nn.Conv2d(in_channels=self.out_channels[1], out_channels=self.out_channels[1],
                                kernel_size=self.kernel_size, dilation=self.dilation, padding=self.padding)
        self.conv23 = nn.Conv2d(in_channels=self.out_channels[1], out_channels=self.out_channels[1],
                                kernel_size=self.kernel_size, dilation=self.dilation, padding=self.padding)

        self.conv31 = nn.Conv2d(in_channels=self.in_channels[2], out_channels=self.out_channels[2],
                                kernel_size=self.kernel_size, dilation=self.dilation, padding=self.padding)
        self.conv32 = nn.Conv2d(in_channels=self.out_channels[2], out_channels=self.out_channels[2],
                                kernel_size=self.kernel_size, dilation=self.dilation, padding=self.padding)
        self.conv33 = nn.Conv2d(in_channels=self.out_channels[2], out_channels=self.out_channels[2],
                                kernel_size=self.kernel_size, dilation=self.dilation, padding=self.padding)

        self.multihead_attn = nn.MultiheadAttention(embed_dim=self.dim, num_heads=self.heads, batch_first=True, )
        self.layer_norm = nn.LayerNorm(self.dim)

        self.flatten = nn.Flatten()
        self.dense = nn.Linear(self.out_channels[2]*dim*self.ppg_channels,self.dense_out)
        self.out = nn.Linear(self.dense_out,1)

        self.dropout1 = nn.Dropout(self.dropout[0])
        self.dropout2 = nn.Dropout(self.dropout[1])
        self.dropout3 = nn.Dropout(self.dropout[2])

    def forward(self, x):
        x = self.emb(x)

        x1 = x[:,:,0:self.ppg_channels,:] #get PPG
        x2 = x[:,:,self.ppg_channels:(self.ppg_channels+3),:] #get acc_x, acc_y, acc_z

        attn_output, attn_output_weights = self.multihead_attn(
            x1.reshape(-1,x1.shape[2]*x1.shape[1],x1.shape[3]), #query vector
            x2.reshape(-1,x2.shape[2]*x2.shape[1],x2.shape[3]), #key vector
            x2.reshape(-1,x2.shape[2]*x2.shape[1],x2.shape[3])) #value vector

        attn_output = self.layer_norm(attn_output)

        flat = self.flatten(attn_output)
        out1 = self.dense(flat)
        out2 = self.out(out1)

        return out2, out1

    def emb(self, x):
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.avg_pool2d(F.relu(self.conv13(x)), kernel_size=self.pooling_size[0], stride=self.pooling_size[0], ceil_mode=True)
        x = self.dropout1(x)

        x = F.relu(self.conv21(x))
        x = F.relu(self.conv22(x))
        x = F.avg_pool2d(F.relu(self.conv23(x)), kernel_size=self.pooling_size[1], stride=self.pooling_size[1], ceil_mode=True)
        x = self.dropout2(x)

        x = F.relu(self.conv31(x))
        x = F.relu(self.conv32(x))
        x = F.avg_pool2d(F.relu(self.conv33(x)), kernel_size=self.pooling_size[2], stride=self.pooling_size[2], ceil_mode=True)
        x = self.dropout3(x)

        return x

In [13]:
class Student(nn.Module):
    def __init__(self, conv_blocks=3, conv_layers=3, in_channels=[1,32,48], out_channels=[32,48,64],
             kernel_size=(1,5), dilation=(1,2), padding=(0,4), dropout=[0.5,0.5,0.5], pooling_size=[(1,4),(1,2),(1,2)],
              heads=4, dim=16, dense_out=32, ppg_channels=1):
        super(Student, self).__init__()

        # hyperparameters
        self.conv_blocks = conv_blocks
        self.conv_layers =conv_layers
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.padding = padding
        self.dropout = dropout
        self.pooling_size = pooling_size
        self.heads = heads
        self.dim = dim
        self.dense_out = dense_out
        self.ppg_channels = ppg_channels


        self.conv11 = nn.Conv2d(in_channels=self.in_channels[0], out_channels=self.out_channels[0],
                                kernel_size=self.kernel_size, dilation=self.dilation, padding=self.padding)
        self.conv12 = nn.Conv2d(in_channels=self.out_channels[0], out_channels=self.out_channels[0],
                                kernel_size=self.kernel_size, dilation=self.dilation, padding=self.padding)
        self.conv13 = nn.Conv2d(in_channels=self.out_channels[0], out_channels=self.out_channels[0],
                                kernel_size=self.kernel_size, dilation=self.dilation, padding=self.padding)

        self.conv21 = nn.Conv2d(in_channels=self.in_channels[1], out_channels=self.out_channels[1],
                                kernel_size=self.kernel_size, dilation=(1,3), padding=(0,3))
        self.conv22 = nn.Conv2d(in_channels=self.out_channels[1], out_channels=self.out_channels[1],
                                kernel_size=self.kernel_size, dilation=(1,3), padding=(0,3))
        self.conv23 = nn.Conv2d(in_channels=self.out_channels[1], out_channels=self.out_channels[1],
                                kernel_size=self.kernel_size, dilation=(1,3), padding=(0,3))

        self.conv31 = nn.Conv2d(in_channels=self.in_channels[2], out_channels=self.out_channels[2],
                                kernel_size=self.kernel_size, dilation=(1,4), padding=(0,4))
        self.conv32 = nn.Conv2d(in_channels=self.out_channels[2], out_channels=self.out_channels[2],
                                kernel_size=self.kernel_size, dilation=(1,4), padding=(0,4))
        self.conv33 = nn.Conv2d(in_channels=self.out_channels[2], out_channels=self.out_channels[2],
                                kernel_size=self.kernel_size, dilation=(1,4), padding=(0,4))

        self.conv4 = nn.Conv2d(in_channels=self.out_channels[2], out_channels=self.out_channels[2],
                                kernel_size=(4,1), padding=(0,0))


        self.flatten = nn.Flatten()
        self.dense = nn.Linear(256,self.dense_out)
        self.out = nn.Linear(self.dense_out,1)

        self.dropout1 = nn.Dropout(self.dropout[0])
        self.dropout2 = nn.Dropout(self.dropout[1])
        self.dropout3 = nn.Dropout(self.dropout[2])


    def forward(self, x):
        x = self.emb(x)
        x = self.conv4(x)

        flat = self.flatten(x)
        out_1 = self.dense(flat)
        out_2 = self.out(out_1)

        return out_2, out_1

    def emb(self, x):
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.avg_pool2d(F.relu(self.conv13(x)), kernel_size=self.pooling_size[0], stride=self.pooling_size[0], ceil_mode=True)
        x = self.dropout1(x)

        x = F.relu(self.conv21(x))
        x = F.relu(self.conv22(x))
        x = F.avg_pool2d(F.relu(self.conv23(x)), kernel_size=self.pooling_size[1], stride=self.pooling_size[1], ceil_mode=True)
        x = self.dropout2(x)

        x = F.relu(self.conv31(x))
        x = F.relu(self.conv32(x))
        x = F.avg_pool2d(F.relu(self.conv33(x)), kernel_size=self.pooling_size[2], stride=self.pooling_size[2], ceil_mode=True)
        x = self.dropout3(x)

        return x

In [14]:
device = torch.device("cuda")

model = Student(in_channels=[1,16,24], out_channels=[16,24,32], kernel_size=(1,3), dilation=(1,2),
                         dropout=[0.1, 0.1, 0.1], padding=(0,2), pooling_size=[(1,4),(1,4),(1,2)])
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

27241

# Training loop with Knowldge Distillation (could be skipped and move to post-processing/evaluation section)

In [18]:
# Get paper's teacher models in case you didn't run training section in prvious notebook
!git clone https://github.com/ounospanas/KD_PULSE.git

Cloning into 'KD_PULSE'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 76 (delta 15), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (76/76), 8.60 MiB | 19.10 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [16]:
!mkdir models_paper/

In [20]:
val_maes = []
test_maes = []
scores = []

for sub_vals in val_sets:

    print('Subjects val are {}'.format(sub_vals))
    for sub_tests in sub_vals:
        print('Subject test is {}'.format(sub_tests))

        tr, val, ts, _train_label, _val_label, _test_label = defineSets(ppg_X, y, subs, sub_vals, sub_tests)

        X_train, X_val, X_test = z_score(tr, val, ts)

        print(X_train.shape)
        print(X_val.shape)
        print(X_test.shape)

        val_len = len(X_val)
        train_len = len(X_train)
        test_len = len(X_test)

        ds_train = ReadyData(X=X_train, y=_train_label, scale_X=False)
        ds_val = ReadyData(X=X_val, y=_val_label, scale_X=False)
        ds_test = ReadyData(X=X_test, y=_test_label, scale_X=False)

        batch_size = 256

        train_set = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
        val_set = DataLoader(ds_val, batch_size=batch_size, shuffle=False)
        test_set = DataLoader(ds_test, batch_size=batch_size, shuffle=False)


        device = torch.device("cuda")

        teacher = Teacher()
        teacher.load_state_dict(torch.load('KD_PULSE/PPG_Dalia_models/best_model_avg_all_acc_last_'+str(sub_tests)+'.pt'))

        model = Student(in_channels=[1,16,24], out_channels=[16,24,32], kernel_size=(1,3), dilation=(1,2),
                         dropout=[0.1, 0.1, 0.1], padding=(0,2), pooling_size=[(1,4),(1,4),(1,2)])

        model.to(device)
        teacher.to(device)

        teacher.eval()

        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-6)
        loss_fn_1 = nn.L1Loss()
        loss_fn_2 = nn.L1Loss()
        loss_fn_3 = nn.MSELoss()

        b = 0.5
        g = 0.6667

        best_loss = 100000000
        best_epoch = 0

        for epoch in range(500):
            tic = time()

            model.train()
            train_loss = 0
            counter = 0


            for batch_idx, (data, target) in enumerate(train_set):
                data, target = Variable(data).float().to(device), Variable(target).float().to(device).reshape(-1,1)
                distil = Variable(target).float().to(device)

                optimizer.zero_grad()
                output, middle = model(data)

                with torch.no_grad():
                    teacher.eval()
                    distil, out2 = teacher(data)

                loss_1 = loss_fn_1(output, target)
                loss_2 = loss_fn_2(output, distil.detach())

                out2 = F.softmax(out2, dim=1)
                middle = F.softmax(middle, dim=1)
                loss_3 = loss_fn_3(out2.detach(), middle)

                loss = g*(b*loss_1 + (1-b)*loss_2) + (1-g)*loss_3

                loss.backward()
                optimizer.step()

                train_loss += loss_fn_1(output, target).item()
                counter +=1
            toc = time()

            print('\nTrain Epoch: {} \tLoss: {:.6f}, time: {:.6f}'.format(epoch, train_loss/counter, toc-tic))

            with torch.no_grad():
                model.eval()
                val_loss = 0
                counter = 0
                custom_loss = 0

                for data, target in val_set:
                    data, target = Variable(data).float().to(device), Variable(target).float().to(device).reshape(-1,1)
                    output,_ = model(data)

                    # sum up batch loss
                    val_loss += loss_fn_1(output, target).item()
                    counter +=1

                val_loss /= counter#len(test_set.dataset)

                #custom_loss /= counter#len(test_set.dataset)
                print('Val set: Average loss: {:.4f}'.format(val_loss))
                if val_loss < best_loss:
                    best_loss = val_loss
                    print("New best loss: {}!!!!!!!!!!!!!!!!".format(best_loss))
                    torch.save(model.state_dict(), 'models_paper/best_model_distil_paper_'+str(sub_tests)+'.pt')
                    best_epoch = epoch
            if epoch>(best_epoch+100):
                break


            with torch.no_grad():
                model.eval()
                test_loss = 0
                counter = 0
                custom_loss = 0

                for data, target in test_set:
                    data, target = Variable(data).float().to(device), Variable(target).float().to(device).reshape(-1,1)
                    output,_ = model(data)

                    # sum up batch loss
                    test_loss += loss_fn_1(output, target).item()
                    counter +=1

                test_loss /= counter#len(test_set.dataset)
                print('Test set: Average loss: {:.4f}'.format(test_loss))


        test_maes.append(test_loss)
        val_maes.append(best_loss)

Subjects val are [5, 6, 7, 8]
Subject test is 5
tensor(-0.0005, dtype=torch.float64) tensor(85.9549, dtype=torch.float64)
tensor(-0.5196, dtype=torch.float64) tensor(0.3501, dtype=torch.float64)
tensor(0.1493, dtype=torch.float64) tensor(0.6334, dtype=torch.float64)
tensor(0.3673, dtype=torch.float64) tensor(0.4099, dtype=torch.float64)
(48721, 1, 4, 256)
(11327, 1, 4, 256)
(4649, 1, 4, 256)

Train Epoch: 0 	Loss: 23.275182, time: 4.581669
Val set: Average loss: 15.6797
New best loss: 15.679690466986763!!!!!!!!!!!!!!!!
Test set: Average loss: 42.1259

Train Epoch: 1 	Loss: 11.470066, time: 4.436953
Val set: Average loss: 14.2903
New best loss: 14.29030736287435!!!!!!!!!!!!!!!!
Test set: Average loss: 40.9488

Train Epoch: 2 	Loss: 10.925641, time: 4.670339
Val set: Average loss: 13.3861
New best loss: 13.386052841610379!!!!!!!!!!!!!!!!
Test set: Average loss: 40.8744


KeyboardInterrupt: 

# Post Processing

In [21]:
# uncomment in case you didn't execute before
# !git clone https://github.com/ounospanas/KD_PULSE.git

In [22]:
val_maes = []
scores = []
all_preds = []
all_targets = []

for sub_vals in val_sets:

    print('Subjects val are {}'.format(sub_vals))
    for sub_tests in sub_vals:
        print('Subjects val are {}'.format(sub_tests))

        tr, val, ts, _train_label, _val_label, _test_label = defineSets(ppg_X, y, subs, sub_vals, sub_tests)

        X_train, X_val, X_test = z_score(tr, val, ts)

        val_len = len(X_val)
        train_len = len(X_train)
        test_len = len(X_test)

        ds_train = ReadyData(X=X_train, y=_train_label, scale_X=False)
        ds_val = ReadyData(X=X_val, y=_val_label, scale_X=False)
        ds_test = ReadyData(X=X_test, y=_test_label, scale_X=False)

        batch_size = 256

        train_set = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
        val_set = DataLoader(ds_val, batch_size=batch_size, shuffle=False)
        test_set = DataLoader(ds_test, batch_size=batch_size, shuffle=False)

        device = torch.device("cuda")

        model = Student(in_channels=[1,16,24], out_channels=[16,24,32], kernel_size=(1,3), dilation=(1,2),
                         dropout=[0.1, 0.1, 0.1], padding=(0,2), pooling_size=[(1,4),(1,4),(1,2)])
        #comment if you ran the training part
        model.load_state_dict(torch.load('KD_PULSE/distilled_paper_models/best_model_distil_paper_'+str(sub_tests)+'.pt'))
        #uncomment if you ran the training part
        #model.load_state_dict(torch.load('models_paper/best_model_distil_paper_'+str(sub_tests)+'.pt'))
        model.to(device)

        optimizer = optim.Adam(model.parameters(), lr=0.001)
        loss_fn = nn.L1Loss()

        with torch.no_grad():
            model.eval()
            correct_test = 0
            test_loss = 0
            counter = 0

            preds_test,_ = model(Variable(torch.Tensor(X_test)).float().to(device))
            preds_test = preds_test.cpu().detach().numpy()
            targets_test = Variable(torch.Tensor(_test_label)).float().to(device).cpu().detach().numpy()
            score_test = np.mean(np.abs(targets_test-preds_test))
            print('TEST MAE:',score_test)



        scores.append(score_test)
        all_preds.append(preds_test)
        all_targets.append(targets_test)
        del X_train, _train_label, X_test, _test_label, X_val, _val_label
        del tr, val, ts

Subjects val are [5, 6, 7, 8]
Subjects val are 5
tensor(-0.0005, dtype=torch.float64) tensor(85.9549, dtype=torch.float64)
tensor(-0.5196, dtype=torch.float64) tensor(0.3501, dtype=torch.float64)
tensor(0.1493, dtype=torch.float64) tensor(0.6334, dtype=torch.float64)
tensor(0.3673, dtype=torch.float64) tensor(0.4099, dtype=torch.float64)
TEST MAE: 7.7606506
Subjects val are 6
tensor(-0.0005, dtype=torch.float64) tensor(85.9549, dtype=torch.float64)
tensor(-0.5196, dtype=torch.float64) tensor(0.3501, dtype=torch.float64)
tensor(0.1493, dtype=torch.float64) tensor(0.6334, dtype=torch.float64)
tensor(0.3673, dtype=torch.float64) tensor(0.4099, dtype=torch.float64)
TEST MAE: 4.809272
Subjects val are 7
tensor(-0.0005, dtype=torch.float64) tensor(85.9549, dtype=torch.float64)
tensor(-0.5196, dtype=torch.float64) tensor(0.3501, dtype=torch.float64)
tensor(0.1493, dtype=torch.float64) tensor(0.6334, dtype=torch.float64)
tensor(0.3673, dtype=torch.float64) tensor(0.4099, dtype=torch.float64)
T

In [23]:
def post_processing(model, x_test, y_test, post=True):

    post_MAE = []

    preds = np.copy(x_test) #it should be model(x_test)

    n = 10 #number_of_examples
    thresh = 10


    for i in range(n,len(preds)):
        if post:
            running_upper = np.mean(preds[(i-n):i])*(100+thresh)/100.0 #1.1
            running_lower = np.mean(preds[(i-n):i])*(100-thresh)/100.0 #0.9

            if preds[i] > running_upper:
                preds[i] = running_upper
                #print(preds[i])
            if preds[i] < running_lower:
                preds[i] = running_lower
                #print(preds[i])

    for j in range(len(preds)):
        mae = np.abs(preds[j]-y_test[j])
        post_MAE.append(mae)

    return np.mean(post_MAE)

In [24]:
post_maes = []
for i in range(15):
    mae = post_processing(model, all_preds[i], all_targets[i])
    post_maes.append(mae)
print(np.mean(post_maes))

4.4944215


In [25]:
# print all scores for the subjects
flattent_subjects = [item for row in val_sets for item in row]
for i, sub in enumerate(flattent_subjects):
  print("Subject {} has MAE: {}".format(sub,post_maes[i]))

Subject 5 has MAE: 7.247197151184082
Subject 6 has MAE: 4.36118221282959
Subject 7 has MAE: 2.6458992958068848
Subject 8 has MAE: 7.385950565338135
Subject 9 has MAE: 8.093338012695312
Subject 10 has MAE: 3.047382354736328
Subject 11 has MAE: 5.015048027038574
Subject 12 has MAE: 6.278552055358887
Subject 13 has MAE: 2.1852259635925293
Subject 14 has MAE: 3.1871988773345947
Subject 15 has MAE: 3.369220495223999
Subject 1 has MAE: 3.8374781608581543
Subject 2 has MAE: 3.596951484680176
Subject 3 has MAE: 2.306058406829834
Subject 4 has MAE: 4.859644889831543
